<a href="https://colab.research.google.com/github/DavidMichaelH/EnchantedJackpot/blob/main/EnchantedJackpot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Require imports for this notebook
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import random

# The Enchated Jackpot

This slots-style game consists of 6 icons. To make our analysis convient we can enumerate them as values $\{0,1,2,3,4,5\}$. The outcome of a single spin results in an random configuration in a 3x4 grid as illustrated below.

|   |   |   |   |
|---|---|---|---|
| 0 | 1 | 3 | 4 |
| 5 | 2 | 0 | 1 |
| 3 | 4 | 5 | 2 |

Each cell takes a random value independently of all other cells and all previous spins. That is, the values of the cells $c_{ij}$ are i.i.d., uniformly distributed over the values $\{0,1,2,3,4,5\}$, and all outcomes are independent between spins.






# Scoring Combinations:


- The game calculates your score based on specific value combinations in each row and column.
- Points are earned through hands, with each hand type having its own bet multiplier.

### Row Multipliers:


- None: No combination - Multiplier: 0
- Single-Double: One pair of matching values - Multiplier: 1/3
- Double-Double: Two pairs of matching values - Multiplier: 5/6
- Triple: Three matching values in a row - Multiplier: 2/3
- Quad: All four values in a row match - Multiplier: 2

### Column Multipliers:

- None: No combination - Multiplier: 0
- Single-Double: One pair of matching values - Multiplier: 1/3
- Triple: Three matching values in a column - Multiplier: 2/3

### Final Score Calculation:

- The final bet multiplier is calculated by summing the multipliers for each row and column.
- The final amount earned by the player for each spin is then computed as `Final Bet Multiplier` $ \times $ `Bet Amount`

In [ ]:
# Pay-Tables for rows and columns expressed as python dictionaries
row_pay_table = {"none":0,"single-double": 1/3, "double-double": 5/6, "triple": 2/3, "quad": 2}
col_pay_table = {"none":0,"single-double": 1/3, "double-double": 0 , "triple": 2/3, "quad": 0}

# Return-to-Player and Variance

We will first estimate the game statistics such as the Return-to-Player (RTP) and variance via monte carlo simulation. Following this we will find congruent results via exact calcultions.


We start by defining a helper function which will determine if any of the relavent combinations are present in a give row/column.

In [ ]:
def analyze_row_col(icons):
    """
    Analyzes a sequence of icons to identify and categorize subsequences of matching icons.

    This function traverses a list of icons and identifies subsequences where the same icon
    occurs consecutively. It categorizes these subsequences based on their length into various
    'hands', such as 'single-double', 'double-double', 'triple', or 'quad'. It also records
    the start and end indices of these subsequences.

    Parameters:
    icons (list of str): A list of strings representing the sequence of icons.

    Returns:
    tuple:
        - hand (str): The highest-value hand identified in the sequence. One of 'none',
          'single-double', 'double-double', 'triple', or 'quad'.
        - indices (list of list of int): List of [start, end] indices for each identified subsequence
          with a length of at least two.

    Example:
    >>> analyze_sequences(['A', 'A', 'B', 'C', 'C', 'C', 'D'])
    ('triple', [[0, 1], [3, 5]])
    """

    hand = "none"
    start = 0
    indices = []

    while start < len(icons):
        end = start
        next_range = [start]
        while end < len(icons) and icons[end] == icons[start]:
            end += 1

        next_range.append(end - 1)
        if end - start >= 2:
            indices.append(next_range)

        sequence_length = end - start

        if sequence_length == 2:
            if hand == "single-double":
                hand = "double-double"
            else:
                hand = "single-double"
        elif sequence_length == 3:
            hand = "triple"
        elif sequence_length == 4:
            hand = "quad"

        start = end

    return hand, indices


# Monte Carlo Simulation

We now generate game value statistics by generating a spin, computing the bet-multiplier, and recording the bet-multiplier in a list.

In [ ]:
random.seed(0)

Trials = 100000
game_values =[]
for trails in tqdm(range(Trials)):

    grid = np.random.randint(1, 6+1, (4, 3))
    grid_transpose = grid.T

    game_value = 0
    for c in range(4):
        hand , _ = analyze_row_col(grid[c])
        game_value += col_pay_table[hand]

    for r in range(3):
        hand , _ = analyze_row_col(grid_transpose[r])
        game_value += row_pay_table[hand]

    game_values.append(game_value)


game_values = np.array(game_values)


### Statistics
We can estimate the RTP as the mean of the game values and as well compute the standard deviation of the game.

In [ ]:
print("Game Stats")
print(f"Mean: {game_values.mean()}")
print(f"Standard deviation: {game_values.std()}")

We can visualize the game statistics as a histogram

In [ ]:
plt.hist(game_values,bins=15)
plt.show()

# Exact Approach

We will now carry out an exact calculation for the sake of completeness and to fortify the confidence in the monte carlo estimates.

In order to carry out an exact analysis we will need the probabilites for the various combinations. We do this by calculating the number of ways certain combinations could occur for rows and columns.


## Row Combinations

For a single row you can have a quad, triple, double-double, or a single-double. Let $C = 6$ denote the number of cell values/icons.

- Quad: $C$
  - There are $C$ possible choices for the common value of the quad.

- Double-Double: $C(C-1)$
  - There are $C$ possible choices for the first pair and $C-1$ for the remaining pair.
- Triple: $ 2C(C-1) $
  - There are $C$ possible choices for the triple pair, $C-1$ for the remaining value, and two ways for the icons to be arranged. That is we can have `AAAB` and `BAAA`.
- Single-Double: $3C(C-1)^2$
  - There are $C$ possible choices for the double pair. Then $C-1$ for each of the remaining values, and three ways for the icons to be arranged.  


# Column Combinations
- Triple: $C$
  - There are $C$ possible choices for the common value of the triple.
- Double: $2C(C-1)$
  - There are $C$ possible choices for the pair, $C-1$ for the remaining values, and $2$ ways for the icons to be arranged.


In [ ]:
# Using our formulas we compute and store the row/column probabilities in the dictionaries.

C = 6

row_probs = {"none":0,"single-double": 0, "double-double": 0, "triple": 0, "quad": 0}
col_probs = {"none":0,"single-double": 0, "double-double": 0, "triple": 0, "quad": 0}

# Row probabilities
row_probs["quad"] = C/C**4
row_probs["triple"] = 2*C*(C-1)/C**4
row_probs["double-double"] = C*(C-1)/C**4
row_probs["single-double"] = (3*C*(C-1)**2)/C**4
row_probs["none"] = 1-sum(row_probs.values())

col_probs["triple"] = C/C**3
col_probs["single-double"] = 2*(C*(C-1))/C**3
col_probs["none"] = 1-sum(col_probs.values())


# Game Value

The value of a game is a random variable $W$ which can be expressed as

$$ W = \sum_{i=1}^3 R_i + \sum_{j=1}^4 C_j $$

where $R_i$ and $C_j$ are the random values associated to the row $i$ and column $j$ respectively. Further $R_i$ and $C_j$ can be expressed as,


$$R_i = \dfrac{1}{3}\text{1}_{\text{Single-Pair at row $i$}} + \dfrac{5}{6}\text{1}_{\text{Double-Pair at row $i$}}+\dfrac{2}{3}\text{1}_{\text{Triple at row $i$}}+2\text{1}_{\text{Quad at row $i$}} $$
$$C_j = \dfrac{1}{3}\text{1}_{\text{Single-Pair at column $j$}}+ \dfrac{2}{3}\text{1}_{\text{Triple at column $j$}} $$

# Expected Game Value

We can compute the expected value of game assuming a bet size of $1$. We will find a result which comports with the monte carlo simulation.

In [ ]:
expected_game_val = 0

for k,p in row_probs.items():
    expected_game_val += 3*p*row_pay_table[k]

for k,p in col_probs.items():
   expected_game_val += 4*p*col_pay_table[k]

print(f"Exact expected value of a single spin: {expected_game_val}")


# Variance Calculations

There is a nice property that allows us to show that the column events and row events are in fact indpendent.

 Indeed let $E_{R}$ denote a general row event (e.g. Row contains Double-Double) and $E_C$ denote an arbitary column event (e.g. Column contains Triple) .  Then if $c$ is the value of their common cell then $\mathbb{P}(E_{R}|c=i) = \mathbb{P}(E_{R}|c  = j)$ and $\mathbb{P}(E_{C}|c=i) = \mathbb{P}(E_{C}|c = j)$ for all $i,j$. As a consequence



 $$ \mathbb{P}(E_{R} ) = \sum_{i=1}^6 \mathbb{P}(E_{R}|c=i)  \mathbb{P}(c = i)= \sum_{i=1}^6 \mathbb{P}(E_{R}|c=j)  \mathbb{P}(c = i) = \mathbb{P}(E_{R}|c=j) \times \sum_{i=1}^6  \mathbb{P}(c = i)  =   \mathbb{P}(E_{R}|c=j)  $$

Using that fact enables us to prove row events are independent of column events. Observe,

$$ \mathbb{P}(E_{R}\cap E_{C} ) = \sum_{i=1}^6 \mathbb{P}(E_{R}\cap E_{C} |c = i)\mathbb{P}(c = i) = \sum_{i=1}^6 \mathbb{P}(E_{R} |c = i) \mathbb{P}(E_{C} |c = i) \mathbb{P}(c = i) $$

$$ = \sum_{i=1}^6 \mathbb{P}(E_{R}) \mathbb{P}(E_{C} ) \mathbb{P}(c = i) =  \mathbb{P}(E_{R}) \mathbb{P}(E_{C} ) $$



Using this we can compute a formula for the variance which we compute below. We do so again assuming a bet size of $1$. We will find a result which comports with the monte carlo simulation.


In [ ]:
variance_game_val = 0

for k1,p1 in row_probs.items():
  for k2,p2 in row_probs.items():
    if k1 != k2:
      variance_game_val += -3*p1*p2*row_pay_table[k1]*row_pay_table[k2]
    else:
      variance_game_val += 3*p1*(1-p1)*row_pay_table[k1]*row_pay_table[k2]

for k1,p1 in col_probs.items():
  for k2,p2 in col_probs.items():
    if k1 != k2:
      variance_game_val += -4*p1*p2*col_pay_table[k1]*col_pay_table[k2]
    else:
      variance_game_val += 4*p1*(1-p1)*col_pay_table[k1]*col_pay_table[k2]

print(f"Theoretical variance of a single game: {np.sqrt(variance_game_val)}")

